In [21]:
import pandas as pd
import jieba

# 一、读取《复仇者联盟4》的好评，中评，差评评论数据

In [22]:
#好评
pos_com =  pd.read_csv('comments_sets/复仇者联盟4_好评.txt',sep = 'ssr',encoding = 'utf-8',header = None)
# sep设置为文档内不包含的内容，否则出错
pos_com.columns = ['comments']

#中评
mid_com  = pd.read_csv('comments_sets/复仇者联盟4_中评.txt',sep = 'ssr',encoding = 'utf-8',header = None)
mid_com.columns = ['comments']


#差评
neg_com = pd.read_csv('comments_sets/复仇者联盟4_差评.txt',sep = 'ssr',encoding = 'utf-8',header = None)
neg_com.columns = ['comments']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if sys.path[0] == '':


In [23]:
pos_com.head()

,comments
0,很好看的。。。
1,终局和开局，漫威宇宙第一季的结尾和第二季的引子，超燃，超豪华，超脑洞，不愧为终局之战！
2,要什么六个字
3,钢铁侠凉了唉
4,好好好好好好好


In [24]:
comment_pos = pos_com['comments']
comment_mid = mid_com['comments']
comment_neg = neg_com['comments']
print('好评的评论数为：',comment_pos.shape)
print('中评的评论数为：',comment_mid.shape)
print('差评的评论数为：',comment_neg.shape)

好评的评论数为： (212489,)
中评的评论数为： (15469,)
差评的评论数为： (25415,)


# 二、去除重复词及水军评论

In [25]:
def condense_1(str):
	# 这里i代表每次处理的字符单位数，如i=1时处理“好好好好”的情况，i=2时处理“很好很好很好”的情况
	# i=1&i=2时用一种处理方式，即当重复数量>2时才进行压缩，因为出现“滔滔不绝”、“美的的确好”
	# 跟“容我思考思考”“这真的真的好看”等不好归为冗余的情况。但当出现3次及以上时基本就是冗余了。
	for i in [1, 2]:
		j = 0
		while j + 3 * i < len(str):
			#判断重复了至少两次
			if str[j: j+i] == str[j+i: j+2*i] and str[j: j+i] == str[j+2*i: j+3*i]:
				k = j+2*i
				while k+i<len(str) and str[j: j+i]==str[k+i: k+2*i]:
					k += i
				str = str[: j+i] + str[k+i:]
			j += 1
	
	# i=3&i=4时用一种处理方式，当重复>1时就进行压缩，因为3个字以上时重复不再构成成语或其他常用语，
	# 基本上即使冗余了。因为大于五个字的重复比较少出现，为了减少算法复杂度可以只处理到i=4。
	for i in [3, 4, 5]:
		j = 0
		while j + 2 * i < len(str):
			#判断重复了至少一次
			if str[j: j+i]==str[j+i: j+2*i]:
				k = j+i
				while k+i<len(str) and str[j: j+i]==str[k+i: k+2*i]:
					k += i
				str = str[: j+i] + str[k+i:]
			j += 1
		i += 1
	
	return str

In [26]:
#总评论的字符长度
print('好评的总字符长度为：',comment_pos.astype('str').apply(lambda x: len(x)).sum())
print('中评的总字符长度为：',comment_mid.astype('str').apply(lambda x: len(x)).sum())
print('差评的总字符长度为：',comment_neg.astype('str').apply(lambda x: len(x)).sum())

好评的总字符长度为： 4515372
中评的总字符长度为： 369595
差评的总字符长度为： 582441


In [27]:
data1_pos = comment_pos.astype('str').apply(lambda x: condense_1(x))  # 去除重复词
data1_mid = comment_mid.astype('str').apply(lambda x: condense_1(x))  # 去除重复词
data1_neg = comment_neg.astype('str').apply(lambda x: condense_1(x))  # 去除重复词

#去除重复词后的字符长度
print('去除重复词后的好评的总字符长度为：',data1_pos.apply(lambda x: len(x)).sum())
print('去除重复词后的中评的总字符长度为：',data1_mid.apply(lambda x: len(x)).sum())
print('去除重复词后的差评的总字符长度为：',data1_neg.apply(lambda x: len(x)).sum())


去除重复词后的好评的总字符长度为： 4371914
去除重复词后的中评的总字符长度为： 363782
去除重复词后的差评的总字符长度为： 573373


In [30]:
#example
print(comment_pos[4])
print(data1_pos[4])

好好好好好好好
好


In [31]:
data2_pos = data1_pos.apply(lambda x: len(x))
data2_mid = data1_mid.apply(lambda x: len(x))
data2_neg = data1_neg.apply(lambda x: len(x))

data3_pos = pd.concat((data1_pos, data2_pos), axis = 1)  # 合并
data3_pos.columns = ['评论','长度']
data3_mid = pd.concat((data1_mid, data2_mid), axis = 1)  # 合并
data3_mid.columns = ['评论','长度']
data3_neg = pd.concat((data1_neg, data2_neg), axis = 1)  # 合并
data3_neg.columns = ['评论','长度']

data3_neg.head()

,评论,长度
0,老套剧情，看了开头就知道结尾！,15
1,垃圾电影！！,6
2,花了钱并没有看 为什么 因为娃去看的,18
3,看完电影对象黄了，就这么简单,14
4,一般下吧啊,5


In [32]:
data4_pos = data3_pos.loc[data3_pos['长度'] > 5, '评论']  # 筛选长度大于5的评论
print('好评非水军评论',data4_pos.shape)

data4_mid = data3_mid.loc[data3_mid['长度'] > 5, '评论']  # 筛选长度大于5的评论
print('中评非水军评论',data4_mid.shape)

data4_neg = data3_neg.loc[data3_neg['长度'] > 5, '评论']  # 筛选长度大于5的评论
print('差评非水军评论',data4_neg.shape)

好评非水军评论 (196761,)
中评非水军评论 (14836,)
差评非水军评论 (24729,)


In [33]:
data4_pos.head()

0                                        很好看的。。。
1    终局和开局，漫威宇宙第一季的结尾和第二季的引子，超燃，超豪华，超脑洞，不愧为终局之战！
2                                         要什么六个字
3                                         钢铁侠凉了唉
5                                        还行吧 还行吧
Name: 评论, dtype: object

# 三、分词

In [34]:
import jieba
#example
list(jieba.cut('我爱北京天安门，天安门前国旗升'))

['我', '爱', '北京', '天安门', '，', '天安门', '前', '国旗', '升']

In [35]:
#好评
data5_pos = data4_pos.apply(lambda x: list(jieba.cut(x)))  
#中评
data5_mid = data4_mid.apply(lambda x: list(jieba.cut(x))) 
#差评
data5_neg = data4_neg.apply(lambda x: list(jieba.cut(x)))

data5_pos.head()

0                                  [很, 好看, 的, 。, 。, 。]
1    [终局, 和, 开局, ，, 漫威, 宇宙, 第一季, 的, 结尾, 和, 第二季, 的, ...
2                                       [要, 什么, 六个, 字]
3                                       [钢铁, 侠凉, 了, 唉]
5                                    [还行, 吧,  , 还行, 吧]
Name: 评论, dtype: object

# 四、去停用词

In [36]:
#读取网上下载的停用词表
stop = pd.read_csv('stoplist.txt', sep='aab', encoding = 'utf-8', header = None)
# sep设置为文档内不包含的内容，否则出错
stop = ['漫威','电影'] + [' ', ''] + list(stop[0]) 	#Pandas自动过滤了空格符，这里手动添加

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [37]:
data6_pos = data5_pos.apply(lambda x: [i for i in x if i not in stop])

data6_mid = data5_mid.apply(lambda x: [i for i in x if i not in stop])

data6_neg = data5_neg.apply(lambda x: [i for i in x if i not in stop])


In [38]:
print('去停用词后的好评字符总长:',data6_pos.apply(lambda x: len(x)).sum(),'减少长度:',\
     data5_pos.apply(lambda x: len(x)).sum() - data6_pos.apply(lambda x: len(x)).sum())

print('去停用词后的中评字符总长:',data6_mid.apply(lambda x: len(x)).sum(),'减少长度:',\
     data5_mid.apply(lambda x: len(x)).sum() - data6_mid.apply(lambda x: len(x)).sum())

print('去停用词后的差评字符总长:',data6_neg.apply(lambda x: len(x)).sum(),'减少长度:',\
     data5_neg.apply(lambda x: len(x)).sum() - data6_neg.apply(lambda x: len(x)).sum())

去停用词后的好评字符总长: 1210519 减少长度: 1540913
去停用词后的中评字符总长: 96313 减少长度: 133895
去停用词后的差评字符总长: 157821 减少长度: 209042


In [39]:
data6_pos.head()

0                                                 [好看]
1    [终局, 开局, 宇宙, 第一季, 结尾, 第二季, 引子, 超燃, 超, 豪华, 超脑, ...
2                                              [六个, 字]
3                                             [钢铁, 侠凉]
5                                             [还行, 还行]
Name: 评论, dtype: object

In [40]:
data6_mid.head()

0          []
1        [服务]
2    [情怀, 支撑]
3         [困]
4    [剧情, 特效]
Name: 评论, dtype: object

In [41]:
data6_neg.head()

0     [老套, 剧情, 开头, 结尾]
1                 [垃圾]
2           [花, 钱, 娃去]
3         [对象, 黄了, 简单]
5    [感觉, 特技, 没什么, 可看]
Name: 评论, dtype: object

In [42]:
#保存好评，中评，差评的经过预处理的数据
data6_pos.to_csv('comments_sets/pre_pos.txt',encoding = 'utf-8', index = False, header = False)

data6_mid.to_csv('comments_sets/pre_mid.txt',encoding = 'utf-8', index = False, header = False)

data6_neg.to_csv('comments_sets/pre_neg.txt',encoding = 'utf-8', index = False, header = False)

In [43]:
#end